This project is the beginning of my exploration into the field of algorithmic trading. I began by learning about the IEX Cloud API, which provides financial data. Since this was my first time, I used the IEX sandbox for free experimentation (with guidance from https://www.youtube.com/watch?v=xfzGZB4HhEE). 

I experiemented with momentum trading, a strategy which evaluates portfolio decisions based on fast growing stocks, with some analysis done to determine potential options with characteristics that could sustain this growth.

The stocks are bathced together in groups of 100 and then sorted by their One-Year Returns. The first approach naively splits the portfolio size equally among the top 50 performing stocks, which is the most basic momentum trading strategy.

The second approach takes a more technical approach, as we look at the returns over One-Year, Six-Months, Three-Months, and One-Month and calculate a score to find higher quality momentum stocks (one that have more consistent returns).

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import math
import requests

In [3]:
# setting up IEX CLOUD API
tickers = pd.read_csv('/Users/sohammahadik/Downloads/sp_500_stocks.csv')
iexcloud_API = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol = 'MSFT'
apiurl = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={iexcloud_API}'
symdata = requests.get(apiurl).json()
symdata

{'companyName': 'Microsoft Corporation',
 'marketcap': 2329493325946,
 'week52high': 316.75,
 'week52low': 204.2,
 'week52highSplitAdjustOnly': 317.15,
 'week52lowSplitAdjustOnly': 202.69,
 'week52change': 0.5093149329824459,
 'sharesOutstanding': 7655356642,
 'float': 0,
 'avg10Volume': 29252581,
 'avg30Volume': 22378285,
 'day200MovingAvg': 277.97,
 'day50MovingAvg': 297.67,
 'employees': 167499,
 'ttmEPS': 8.33,
 'ttmDividendRate': 2.276570430497399,
 'dividendYield': 0.007557610285279207,
 'nextDividendDate': '2021-11-15',
 'exDividendDate': '2021-08-11',
 'nextEarningsDate': '2021-10-11',
 'peRatio': 38.51804596576849,
 'beta': 1.167308645137282,
 'maxChangePercent': 12.410404189951349,
 'year5ChangePercent': 4.804961906800532,
 'year2ChangePercent': 1.2005136469994389,
 'year1ChangePercent': 0.52000095337558,
 'ytdChangePercent': 0.356720666351978,
 'month6ChangePercent': 0.2794352659618479,
 'month3ChangePercent': 0.132372588045196,
 'month1ChangePercent': -0.016981578027496357,

In [4]:
# divide stocks into groups for batch API calls
def group(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

symgroups = list(group(tickers['Ticker'], 100))
symstrings = []
for i in range(0, len(symgroups)):
    symstrings.append(','.join(symgroups[i]))
    
cols = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']


In [5]:
df = pd.DataFrame(columns = cols)

for symstr in symstrings:
    batchAPI = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symstr}&types=price,stats&token={iexcloud_API}'
    data = requests.get(batchAPI).json()
    for sym in symstr.split(','):
        df = df.append(
            pd.Series(
            [
                sym,
                data[sym]['price'],
                data[sym]['stats']['year1ChangePercent'],
                'N/A'
            ], 
            index = cols,
            ),
            ignore_index = True
        )
        
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,179.28,0.799754,N/A
1,AAL,22.10,0.787193,N/A
2,AAP,223.99,0.482038,N/A
3,AAPL,149.20,0.389541,N/A
4,ABBV,110.71,0.293749,N/A
...,...,...,...,...
500,YUM,126.22,0.427701,N/A
501,ZBH,154.20,0.133401,N/A
502,ZBRA,563.28,1.256383,N/A
503,ZION,61.45,1.21802,N/A


In [6]:
df.sort_values('One-Year Price Return', ascending=False, inplace=True)
df = df[:50]
#df.reset_index(inplace=True)

def set_size():
    global size
    size = input('Input the asset size your portfolio:')
    try:
        float(size)
    except ValueError:
        print('Please input a numerical value.')
        input('Input the asset size your portfolio:')


In [7]:
# Naive model - evenly divide assets among stocks
set_size()
position = float(size)/len(df.index)
for i in range(0, len(df)):
    df.loc[i, 'Number of Shares to Buy'] = math.floor(position/df.loc[i, 'Price'])
    
df

Input the asset size your portfolio:1000000


KeyError: 0

In [27]:
indepth_cols = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'Momentum Score'
]

indepth_df = pd.DataFrame(columns = indepth_cols)
indepth_df

for symstr in symstrings:
    batchAPI = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symstr}&types=price,stats&token={iexcloud_API}'
    data = requests.get(batchAPI).json()
    for sym in symstr.split(','):
        indepth_df = indepth_df.append(
            pd.Series(
            [
                sym,
                data[sym]['price'],
                'N/A',
                data[sym]['stats']['year1ChangePercent'],
                'N/A',
                data[sym]['stats']['month6ChangePercent'],
                'N/A',
                data[sym]['stats']['month3ChangePercent'],
                'N/A',
                data[sym]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ], 
            index = indepth_cols,
            ),
            ignore_index = True
        )

indepth_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Momentum Score
0,A,174.493,N/A,0.79822,N/A,0.432098,N/A,0.183515,N/A,0.016516,N/A,N/A
1,AAL,22.150,N/A,0.752943,N/A,-0.062919,N/A,-0.059889,N/A,0.095044,N/A,N/A
2,AAP,219.610,N/A,0.481685,N/A,0.213829,N/A,0.075267,N/A,0.036579,N/A,N/A
3,AAPL,149.841,N/A,0.396988,N/A,0.206067,N/A,0.101521,N/A,-0.019565,N/A,N/A
4,ABBV,108.540,N/A,0.306723,N/A,0.048456,N/A,-0.049581,N/A,-0.102894,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.790,N/A,0.42496,N/A,0.17416,N/A,0.076228,N/A,-0.071554,N/A,N/A
501,ZBH,157.155,N/A,0.136355,N/A,-0.034465,N/A,-0.085624,N/A,0.034828,N/A,N/A
502,ZBRA,553.700,N/A,1.225397,N/A,0.19776,N/A,0.105536,N/A,-0.044855,N/A,N/A
503,ZION,62.549,N/A,1.179872,N/A,0.126114,N/A,0.130189,N/A,0.08347,N/A,N/A


In [28]:
intervals = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for r in indepth_df.index:
    for interval in intervals:
        delta = f'{interval} Price Return'
        percent = f'{interval} Return Percentile'
        if indepth_df.loc[r, delta] == None:
            indepth_df.loc[r, delta] = 0.0

for r in indepth_df.index:
    for interval in intervals:
        delta = f'{interval} Price Return'
        percent = f'{interval} Return Percentile'
        indepth_df.loc[r, percent] = stats.percentileofscore(indepth_df[delta], indepth_df.loc[r, delta]) 

        

indepth_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Momentum Score
0,A,174.493,N/A,0.79822,80.594059,0.432098,96.039604,0.183515,91.881188,0.016516,67.128713,N/A
1,AAL,22.150,N/A,0.752943,77.227723,-0.062919,12.277228,-0.059889,20.0,0.095044,93.267327,N/A
2,AAP,219.610,N/A,0.481685,55.049505,0.213829,73.465347,0.075267,70.49505,0.036579,78.415842,N/A
3,AAPL,149.841,N/A,0.396988,45.544554,0.206067,73.069307,0.101521,78.019802,-0.019565,42.772277,N/A
4,ABBV,108.540,N/A,0.306723,36.237624,0.048456,37.029703,-0.049581,23.762376,-0.102894,3.960396,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.790,N/A,0.42496,49.70297,0.17416,66.732673,0.076228,71.287129,-0.071554,11.089109,N/A
501,ZBH,157.155,N/A,0.136355,17.029703,-0.034465,16.435644,-0.085624,12.871287,0.034828,77.623762,N/A
502,ZBRA,553.700,N/A,1.225397,94.059406,0.19776,71.287129,0.105536,79.207921,-0.044855,25.148515,N/A
503,ZION,62.549,N/A,1.179872,92.871287,0.126114,57.029703,0.130189,83.168317,0.08347,91.287129,N/A


In [29]:
from statistics import mean

for r in indepth_df.index:
    momPercent = []
    for interval in intervals:
        momPercent.append(indepth_df.loc[r, f'{interval} Return Percentile'])
    indepth_df.loc[r, 'Momentum Score'] = mean(momPercent)

indepth_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Momentum Score
0,A,174.493,N/A,0.79822,80.594059,0.432098,96.039604,0.183515,91.881188,0.016516,67.128713,83.910891
1,AAL,22.150,N/A,0.752943,77.227723,-0.062919,12.277228,-0.059889,20.0,0.095044,93.267327,50.693069
2,AAP,219.610,N/A,0.481685,55.049505,0.213829,73.465347,0.075267,70.49505,0.036579,78.415842,69.356436
3,AAPL,149.841,N/A,0.396988,45.544554,0.206067,73.069307,0.101521,78.019802,-0.019565,42.772277,59.851485
4,ABBV,108.540,N/A,0.306723,36.237624,0.048456,37.029703,-0.049581,23.762376,-0.102894,3.960396,25.247525
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.790,N/A,0.42496,49.70297,0.17416,66.732673,0.076228,71.287129,-0.071554,11.089109,49.70297
501,ZBH,157.155,N/A,0.136355,17.029703,-0.034465,16.435644,-0.085624,12.871287,0.034828,77.623762,30.990099
502,ZBRA,553.700,N/A,1.225397,94.059406,0.19776,71.287129,0.105536,79.207921,-0.044855,25.148515,67.425743
503,ZION,62.549,N/A,1.179872,92.871287,0.126114,57.029703,0.130189,83.168317,0.08347,91.287129,81.089109


In [30]:
indepth_df.sort_values('Momentum Score', ascending=False, inplace=True)
indepth_df = indepth_df[:50]
indepth_df.reset_index(inplace=True, drop=True)
indepth_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Momentum Score
0,MCHP,167.654,N/A,2.486988,99.80198,1.251989,100.0,1.324883,100.0,1.234637,100.0,99.950495
1,PWR,119.218,N/A,1.373137,96.237624,0.454246,96.237624,0.33726,98.613861,0.221935,99.207921,97.574257
2,LB,81.450,N/A,2.336279,99.60396,0.85214,99.80198,0.21562,95.445545,0.081329,91.089109,96.485149
3,IT,327.200,N/A,1.646389,97.821782,0.790504,99.60396,0.377302,99.405941,0.072691,88.514851,96.336634
4,DVN,34.290,N/A,2.91674,100.0,0.53528,98.019802,0.137592,85.148515,0.210984,99.009901,95.544554
5,NTAP,93.529,N/A,1.3473,95.841584,0.382294,92.673267,0.151148,87.326733,0.156304,97.821782,93.415842
6,IPG,39.620,N/A,1.488771,96.633663,0.393266,93.861386,0.2173,96.039604,0.047329,82.772277,92.326733
7,SIVB,663.820,N/A,1.855604,98.613861,0.299439,85.346535,0.152326,87.524752,0.151749,97.623762,92.277228
8,MSCI,660.702,N/A,0.943268,87.326733,0.561876,98.415842,0.250194,97.425743,0.054108,84.752475,91.980198
9,ALGN,755.500,N/A,1.365918,96.039604,0.402834,94.653465,0.207741,94.059406,0.043964,81.386139,91.534653


In [31]:
set_size()

Input the asset size your portfolio:1000000


In [32]:
posSize = float(size)/len(indepth_df.index)
for i in indepth_df.index:
    indepth_df.loc[i, 'Number of Shares to Buy'] = math.floor(posSize/indepth_df.loc[i, 'Price'])
    
indepth_df

/Users/sohammahadik/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,Momentum Score
0,MCHP,167.654,119,2.486988,99.80198,1.251989,100.0,1.324883,100.0,1.234637,100.0,99.950495
1,PWR,119.218,167,1.373137,96.237624,0.454246,96.237624,0.33726,98.613861,0.221935,99.207921,97.574257
2,LB,81.450,245,2.336279,99.60396,0.85214,99.80198,0.21562,95.445545,0.081329,91.089109,96.485149
3,IT,327.200,61,1.646389,97.821782,0.790504,99.60396,0.377302,99.405941,0.072691,88.514851,96.336634
4,DVN,34.290,583,2.91674,100.0,0.53528,98.019802,0.137592,85.148515,0.210984,99.009901,95.544554
5,NTAP,93.529,213,1.3473,95.841584,0.382294,92.673267,0.151148,87.326733,0.156304,97.821782,93.415842
6,IPG,39.620,504,1.488771,96.633663,0.393266,93.861386,0.2173,96.039604,0.047329,82.772277,92.326733
7,SIVB,663.820,30,1.855604,98.613861,0.299439,85.346535,0.152326,87.524752,0.151749,97.623762,92.277228
8,MSCI,660.702,30,0.943268,87.326733,0.561876,98.415842,0.250194,97.425743,0.054108,84.752475,91.980198
9,ALGN,755.500,26,1.365918,96.039604,0.402834,94.653465,0.207741,94.059406,0.043964,81.386139,91.534653
